In [1]:
# Translated to .py by Monique A. Stinson
# 04.01.2017
# Adapted to PandasBiogeme by Michel Bierlaire
# Thu Nov  1 17:42:56 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.models import logcnl_avail
from biogeme.expressions import Beta, DefineVariable, log

df = pd.read_csv("telephone.dat",'\t')
database = db.Database("telephone",df)
pd.options.display.float_format = '{:.3g}'.format
globals().update(database.variables)


#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
ASC_BM	 = Beta('ASC_BM',0,None,None,0)
ASC_EF	 = Beta('ASC_EF',0,None,None,0)
ASC_LF	 = Beta('ASC_LF',0,None,None,0)
ASC_MF	 = Beta('ASC_MF',0,None,None,0)
ASC_SM	 = Beta('ASC_SM',0,None,None,1)
B_COST	 = Beta('B_COST',0,None,None,0)
#
# parameters relevant to the nests
N_FLAT = Beta('N_FLAT',1,1,None,0)
N_MEAS = Beta('N_MEAS',1,1,None,0)
#
a_FLAT_LF = Beta('a_FLAT_LF',0.5,0,1,0)
a_MEAS_LF = 1 - a_FLAT_LF
a_FLAT_EF = Beta('a_FLAT_EF',1,0,1,1)
a_FLAT_MF = Beta('a_FLAT_MF',1,0,1,1)
a_MEAS_SM = Beta('a_FLAT_SM',1,0,1,1)
a_MEAS_BM = Beta('a_MEAS_BM',1,0,1,1)
#

# Define here arithmetic expressions for names that are not directly
# available from the data

logcostBM  = DefineVariable('logcostBM', log(cost1),database)
logcostSM  = DefineVariable('logcostSM', log(cost2),database)
logcostLF  = DefineVariable('logcostLF', log(cost3),database)
logcostEF  = DefineVariable('logcostEF', log(cost4),database)
logcostMF  = DefineVariable('logcostMF', log(cost5),database)

#Utilities
V_BM = ASC_BM + B_COST * logcostBM
V_SM = ASC_SM + B_COST * logcostSM
V_LF = ASC_LF + B_COST * logcostLF
V_EF = ASC_EF + B_COST * logcostEF
V_MF = ASC_MF + B_COST * logcostMF

V = {1: V_BM, 2: V_SM, 3: V_LF, 4: V_EF, 5: V_MF}
avail = {1: avail1, 2: avail2, 3: avail3, 4: avail4, 5: avail5}

#Definitions of nests
alpha_N_FLAT = {1: 0, 2: 0, 3: a_FLAT_LF, 4: a_FLAT_EF, 5: a_FLAT_MF}
alpha_N_MEAS = {1: a_MEAS_BM, 2: a_MEAS_SM, 3: a_MEAS_LF, 4: 0, 5: 0}

nest_N_FLAT = N_FLAT, alpha_N_FLAT
nest_N_MEAS = N_MEAS, alpha_N_MEAS

nests = nest_N_FLAT, nest_N_MEAS

# CNL with variable alphas
logprob = logcnl_avail(V, avail, nests, choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "MEV_Tel_CNL_var"
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")


           Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
ASC_BM     -0.36    0.129    -2.8  0.00513         0.127        -2.84   
ASC_EF     0.813     0.39    2.08   0.0372         0.384         2.12   
ASC_LF     0.943    0.146    6.45 1.11e-10         0.133          7.1   
ASC_MF      1.35     0.33    4.08 4.52e-05         0.282         4.78   
B_COST     -1.43    0.329   -4.36 1.31e-05         0.293         -4.9   
N_FLAT      2.42    0.854    2.84  0.00455         0.958         2.53   
N_MEAS      2.17    0.714    3.05  0.00231         0.704         3.09   
a_FLAT_LF  0.907    0.192    4.71 2.44e-06         0.131         6.91   

           Rob. p-value  
ASC_BM          0.00448  
ASC_EF           0.0342  
ASC_LF         1.24e-12  
ASC_MF         1.78e-06  
B_COST         9.82e-07  
N_FLAT           0.0114  
N_MEAS          0.00202  
a_FLAT_LF      5.01e-12  
Nbr of observations: 434
LL(0) =    -560.250
LL(beta) = -473.165
rho bar square = 0.141
Output file: MEV_Tel

In [2]:
pandasResults

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_BM,-0.36,0.129,-2.8,0.00513,0.127,-2.84,0.00448
ASC_EF,0.813,0.39,2.08,0.0372,0.384,2.12,0.0342
ASC_LF,0.943,0.146,6.45,1.11e-10,0.133,7.1,1.24e-12
ASC_MF,1.35,0.33,4.08,4.52e-05,0.282,4.78,1.78e-06
B_COST,-1.43,0.329,-4.36,1.31e-05,0.293,-4.9,9.82e-07
N_FLAT,2.42,0.854,2.84,0.00455,0.958,2.53,0.0114
N_MEAS,2.17,0.714,3.05,0.00231,0.704,3.09,0.00202
a_FLAT_LF,0.907,0.192,4.71,2.44e-06,0.131,6.91,5.01e-12
